 ## **Criação do dataset**

Os dados fornecidos para este trabalho estarão no formato JSON e serão disponibilizados a partir da plataforma de conhecimento sobre cancro gástrico desenvolvido no grupo BioSystems.

**Aceder à API e analisar a estrutura dos dados**

Iniciamos o projeto por aceder á API (https://gcplatform.bio.di.uminho.pt/api/pub/swagger-ui/index.html?configUrl=/api/pub/v3/api-docs/swagger-config#/pageable-publications-controller/findPubmedPubsByQuery) e analisar os dados relativos ao cancro do estômago.

Após uma análise verificamos que a estrutura dos objetos era a seguinte:

{
    id (string)
    idGenerator (string)
    source (string)
    createdDate (string date-time)
    lastModifiedDate (string date-time)
    title (string)
    summary (string)
    content (string)
    xdbRefs (uniqueItems: true string)
    meshTerms (uniqueItems: true string)
    keywords (uniqueItems: true string)
    
}

Também verificamos que da estrutura das páginas JSON da API apenas nos interessava o array "content", que continha os objetos com a estrutura em cima representada e com a informação que necessitavamos.


**Recolher informação da API para um ficheiro json**

De seguida, foi necessário recolher a informação importante da API para um ficheiro JSON.
Para tal, utilizamos três bibliotecas. A biblioteca **urllib.parse**, para manipularmos o url, a biblioteca **requests**, que é uma biblioteca HTTP necessária para recebermos a informação da API, e, a biblioteca **json**, para manipular os ficheiros json.

Posto isto, criamos um loop que efetuas pedidos sobre as 145 páginas existentes na API sobre o cancro do estômago e que coloca, a informação do content de cada página, numa pasta.

In [ ]:
import urllib.parse, requests, json
import os

os.makedirs('pages')

#Rescolher a informação da API pub
for n_page in range(146):
    main_api = 'https://gcplatform.bio.di.uminho.pt/api/pub/page/find/pubmedindexed/gastric%20cancer?page={}&size=1000&sort=string'.format(n_page)
    address = 'lhr'
    url = main_api + urllib.parse.urlencode({'address': address})
    json_data = requests.get(url).json()

    #Mudar o nome do ficheiro
    filename = "pages\\page_{}_gastric_cancer.json".format(n_page)
  

    #Escrever a informação num ficheiro json
    with open(filename, 'a') as json_file:
        json.dump(json_data["content"], json_file, indent=4, sort_keys=True) #Formata o json file, o indent e sort_keys

Por fim, foi necessário juntar todas as páginas numa única lista de objetos, ou seja, num ficheiro json com todos os objetos.

In [ ]:
import json
from os import listdir
from os.path import isfile, join

lista=[]  

files = [f for f in listdir("pages") if isfile(join("pages", f))]

for file in files:
    f = open('pages/'+file)
    data = json.load(f) 
    for i in data: lista.append(i)
    f.close()

with open('info_gastric_cancer.json', 'w') as json_file:
    json.dump(lista, json_file,indent=1)

**Converter o ficheiro json em objetos python**

Inicialmente, criamos uma classe Info com quatro métodos, sendo dois os mais importantes. O primeiro método, o init, é executado quando criamos uma instância de um objeto. O segundo método, o to dict, é utilizado para converter os objetos que guardamos na lista num dicionário.

Inicialmente, criamos uma classe Info com quatro métodos, sendo dois os mais importantes. O primeiro método, o init, é executado quando criamos uma instância de um objeto. O segundo método, o to dict, é utilizado para converter os objetos que guardamos na lista num dicionário.

In [ ]:
import json
import pandas as pd


class Info:
    def __init__(self, id=None, idGenerator=None, source=None, createdDate=None, lastModifiedDate=None, title=None, summary=None, content=None, xdbRefs=None, meshTerms=None, keywords=None):
        self.id = id
        self.idGenerator = idGenerator
        self.source = source
        self.createdDate = createdDate
        self.lastModifiedDate = lastModifiedDate
        self.title = title
        self.summary = summary
        self.content = content
        self.xdbRefs = xdbRefs
        self.meshTerms = meshTerms
        self.keywords = keywords


    @classmethod
    def from_json(cls, json_string):
        json_dict = json.loads(json_string)
        return cls(**json_dict)


    def __repr__(self):
        #return str(self)
        return f'{self.id}'
        
    def to_dict(self):
        return {
            'id': self.id,
            'title': self.title,
            'summary':self.summary,
        }

Posto isto, foi necessário ler o ficheiro json com todos os objetos e colocar esses objetos numa lista.

Tendo já uma lista com os objetos é necessário colocar estes objetos num dataframe, recorrendo à biblioteca pandas.

Por fim, foi exportado o dataframe para um ficheiro csv.

In [ ]:
info_list = []
with open('info_gastric_cancer.json', 'r') as json_file:
    info_data = json.loads(json_file.read())
    for i in info_data: 
        info_list.append(Info(**i))

df = pd.DataFrame.from_records([a.to_dict() for a in info_list])
df.index = df["id"]
del df["id"]

df.to_csv('dataset_gastric_cancer.csv', sep=",")

Agora iremos obter 50 publicações por cada sub doença através da API das publicações.

Em primeiro lugar vamos obter todos os DOID, os identificadores de todas as sub doenças.

De seguida obtemos a informação de cada uma das sub doenças de modo a que de seguida fosse possível obtermos as palavras chaves de cada sub doenças.

Assim foi possível obter os documentos através das palavras chaves que representam as sub doenças 

In [ ]:
import urllib.request, json 
from urllib.parse import quote
import re
import csv


def get_disease_pub_id(names):
  url_request="https://gcplatform.bio.di.uminho.pt/api/pub/elastic/findid/content/terms?"
  for name in names:
    url_request+="terms="+name+"&"
  url_request+="page=0&size=50"#get only 50
  url_request = re.sub(r"\s+", '%20', url_request)
  
  lista=[]
  with urllib.request.urlopen(url_request) as url:  
    pub_ids_response = json.loads(url.read().decode())
    pub_ids=pub_ids_response["content"] 
    for pub_id in pub_ids:
        pubmed=pub_id["id"].split(":")
        lista.append(pubmed[1])
    return lista    


def get_disease_pubs(lista,disease_pub_id,doid):
    for pubid in disease_pub_id:
        with urllib.request.urlopen('https://gcplatform.bio.di.uminho.pt/api/pub/ids?id=pubmed%3A'+pubid) as url:  
            pubs = json.loads(url.read().decode())
            for pub in pubs:
                objecto={"doid": doid.split(":")[1]}
                if "id" in pub: objecto["id"]=pub["id"].split(":")[1]
                else: objecto["id"]=""

                if "title" in pub: objecto["title"]=pub["title"]
                else: objecto["title"]=""
                
                if "summary" in pub: objecto["summary"]=pub["summary"]
                else: objecto["summary"]=""

                lista.append(objecto)
    return lista

#get all sub diseases doids
with urllib.request.urlopen('https://gcplatform.bio.di.uminho.pt/api/dise/allsubdiseids/DOID%3A10534') as url:  
    doids = json.loads(url.read().decode())
    i=0
    lista=[]
    #iterate over all doids
    for doid in doids:
      id = doid.split(":")[1] #obter id da string "DOID:4716"
      with urllib.request.urlopen('https://gcplatform.bio.di.uminho.pt/api/dise/id/DOID%3A'+id) as url_disease:
        disease = json.loads(url_disease.read().decode())
        names=disease["names"] #get names of disease

        disease_pub_id = get_disease_pub_id(names)
        lista=get_disease_pubs(lista,disease_pub_id,doid)
        print(i)
        i=i+1

with open('personal.json', 'w') as json_file:
    json.dump(lista, json_file, indent=4)